# Harmony and GDAL Subsetter 
## Regression Tests for ASF Collections

## Environment Selection:

In [1]:
SUPPORTED_ENVS = ['sit', 'uat', 'prod']
global env
env = input("Input the environment (sit, uat, or prod) or 'Enter' for uat: ") or 'uat'
if env.lower() not in SUPPORTED_ENVS:
            raise Exception(f'{env} is not a supported environment (supported envs: {SUPPORTED_ENVS})')
print("Environment: ", env)
if env.lower() == 'sit':
    print('Connect to the NASA VPN before running requests against the sit environment.')

Input the environment (sit, uat, or prod) or 'Enter' for uat: 
Environment:  uat


## Regression tests for the AVNIR-2 collection:

In [4]:
import requests
import avnir_info
import get_base_url_info
import get_product_info
import get_nc_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
collection = 'avnir'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
avnir = avnir_info.avnir_info()
for num_requests in avnir:    
    if num_requests['test_mode'] == 'standard_subsetting' or num_requests['test_mode'] == 'filetype':
        harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
        outfile = env + num_requests['outfile']
        print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
        print(num_requests['message'])
        r = requests.get(harmony_url)
    elif num_requests['test_mode'] == 'shapefile':
        harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url']
        outfile = env + num_requests['outfile']
        print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
        print(num_requests['message'])
        path = num_requests['shape_path']
        file = num_requests['shapefile_input']
        gid = ''
        if env == 'prod':
            gid = num_requests['prod_gid']
        elif env == 'uat' or env == 'sit':
            gid = num_requests['uat_gid']
        data = {'format': num_requests['outfile_format'],'granuleID': gid}
        files = {
            'shapefile': (
            file,
            open(path + file,'rb'),
            num_requests['post_type']
            ),
            }
        r = requests.post(harmony_url, files=files, data=data)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext[0:] == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_avnir_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    elif file_ext[2:] == 'nc':
        product_info = get_nc_info.get_nc_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_avnir_nc_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)   
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_avnir_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)        
        
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)

Running request 1: https://harmony.uat.earthdata.nasa.gov/C1233629671-ASF/ogc-api-coverages/1.0.0/collections/Band1%2CBand2/coverage/rangeset?subset=lat(-.05:0.25)&subset=lon(-51.0:-50.75)&format=image%2Ftiff&granuleID=G1236469528-ASF
Subset across the Equator. Variable subset bands 1 & 2
Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_avnir_1.py::test_avnir_1_cs PASSED                                  [  9%]
test_avnir_1.py::test_avnir_1_projection PASSED                          [ 18%]
test_avnir_1.py::test_avnir_1_proj_epsg PASSED                           [ 27%

Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_avnir_6.py::test_avnir_6_cs PASSED                                  [  9%]
test_avnir_6.py::test_avnir_6_projection PASSED                          [ 18%]
test_avnir_6.py::test_avnir_6_proj_epsg PASSED                           [ 27%]
test_avnir_6.py::test_avnir_6_gcs PASSED                                 [ 36%]
test_avnir_6.py::test_avnir_6_gcs_epsg PASSED                            [ 45%]
test_avnir_6.py::test_avnir_6_authority PASSED                           [ 54%]
test_avnir_6.py::test_avnir_6_subset PASSED     

collected 11 items                                                             

test_avnir_12.py::test_avnir_12_cs PASSED                                [  9%]
test_avnir_12.py::test_avnir_12_projection PASSED                        [ 18%]
test_avnir_12.py::test_avnir_12_proj_epsg PASSED                         [ 27%]
test_avnir_12.py::test_avnir_12_gcs PASSED                               [ 36%]
test_avnir_12.py::test_avnir_12_gcs_epsg PASSED                          [ 45%]
test_avnir_12.py::test_avnir_12_authority PASSED                         [ 54%]
test_avnir_12.py::test_avnir_12_subset PASSED                            [ 63%]
test_avnir_12.py::test_avnir_12_bands PASSED                             [ 72%]
test_avnir_12.py::test_avnir_12_variables PASSED                         [ 81%]
test_avnir_12.py::test_avnir_12_xy_size PASSED                           [ 90%]
test_avnir_12.py::test_avnir_12_identical_to_reference_image PASSED      [100%]

============================== 11 pass

test_avnir_16.py::test_avnir_16_bands PASSED                             [ 72%]
test_avnir_16.py::test_avnir_16_variables PASSED                         [ 81%]
test_avnir_16.py::test_avnir_16_xy_size PASSED                           [ 90%]
test_avnir_16.py::test_avnir_16_identical_to_reference_image PASSED      [100%]

============================== 11 passed in 0.08s ==============================
Running request 17: https://harmony.uat.earthdata.nasa.gov/C1233629671-ASF/ogc-api-coverages/1.0.0/collections/Band2/coverage/rangeset
Test of variable (band 2) and spatial subsetting for a zip file containing a shapefile of a cw polygon that spans the equator
Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter

Running request 21: https://harmony.uat.earthdata.nasa.gov/C1233629671-ASF/ogc-api-coverages/1.0.0/collections/Band1%2CBand3/coverage/rangeset
Test of variable (bands 1 and 3) and spatial subsetting for a Geojson file containing a shapefile of a ccw polygon that spans the equator
Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_avnir_21.py::test_avnir_21_cs PASSED                                [  9%]
test_avnir_21.py::test_avnir_21_projection PASSED                        [ 18%]
test_avnir_21.py::test_avnir_21_proj_epsg PASSED                         [ 27%]
test_av

collected 11 items                                                             

test_avnir_26.py::test_avnir_26_cs PASSED                                [  9%]
test_avnir_26.py::test_avnir_26_projection PASSED                        [ 18%]
test_avnir_26.py::test_avnir_26_proj_epsg PASSED                         [ 27%]
test_avnir_26.py::test_avnir_26_gcs PASSED                               [ 36%]
test_avnir_26.py::test_avnir_26_gcs_epsg PASSED                          [ 45%]
test_avnir_26.py::test_avnir_26_authority PASSED                         [ 54%]
test_avnir_26.py::test_avnir_26_subset PASSED                            [ 63%]
test_avnir_26.py::test_avnir_26_bands PASSED                             [ 72%]
test_avnir_26.py::test_avnir_26_variables PASSED                         [ 81%]
test_avnir_26.py::test_avnir_26_xy_size PASSED                           [ 90%]
test_avnir_26.py::test_avnir_26_identical_to_reference_image PASSED      [100%]

============================== 11 pass

test_avnir_16.py::test_avnir_16_gcs_epsg PASSED                          [ 24%]
test_avnir_16.py::test_avnir_16_authority PASSED                         [ 24%]
test_avnir_16.py::test_avnir_16_subset PASSED                            [ 25%]
test_avnir_16.py::test_avnir_16_bands PASSED                             [ 25%]
test_avnir_16.py::test_avnir_16_variables PASSED                         [ 25%]
test_avnir_16.py::test_avnir_16_xy_size PASSED                           [ 26%]
test_avnir_16.py::test_avnir_16_identical_to_reference_image PASSED      [ 26%]
test_avnir_17.py::test_avnir_17_cs PASSED                                [ 27%]
test_avnir_17.py::test_avnir_17_projection PASSED                        [ 27%]
test_avnir_17.py::test_avnir_17_proj_epsg PASSED                         [ 27%]
test_avnir_17.py::test_avnir_17_gcs PASSED                               [ 28%]
test_avnir_17.py::test_avnir_17_gcs_epsg PASSED                          [ 28%]
test_avnir_17.py::test_avnir_17_authorit

test_avnir_24.py::test_avnir_24_identical_to_reference_image PASSED      [ 65%]
test_avnir_26.py::test_avnir_26_cs PASSED                                [ 65%]
test_avnir_26.py::test_avnir_26_projection PASSED                        [ 66%]
test_avnir_26.py::test_avnir_26_proj_epsg PASSED                         [ 66%]
test_avnir_26.py::test_avnir_26_gcs PASSED                               [ 67%]
test_avnir_26.py::test_avnir_26_gcs_epsg PASSED                          [ 67%]
test_avnir_26.py::test_avnir_26_authority PASSED                         [ 67%]
test_avnir_26.py::test_avnir_26_subset PASSED                            [ 68%]
test_avnir_26.py::test_avnir_26_bands PASSED                             [ 68%]
test_avnir_26.py::test_avnir_26_variables PASSED                         [ 69%]
test_avnir_26.py::test_avnir_26_xy_size PASSED                           [ 69%]
test_avnir_26.py::test_avnir_26_identical_to_reference_image PASSED      [ 69%]
test_avnir_4.py::test_avnir_4_cs PASSED 

## Regression tests for the ALOS_PALSAR_LEVEL2.2 collection:

In [2]:
import requests
import alos_psr_l22_info
import get_base_url_info
import get_product_info
import get_nc_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
collection = 'alos_l22'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
alos_psr_l22 = alos_psr_l22_info.alos_psr_l22_info()
for num_requests in alos_psr_l22:    
    if num_requests['test_mode'] == 'standard_subsetting' or num_requests['test_mode'] == 'filetype':
        harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
        outfile = env + num_requests['outfile']
        print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
        print(num_requests['message'])
        r = requests.get(harmony_url)
    elif num_requests['test_mode'] == 'shapefile':
        harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url']
        outfile = env + num_requests['outfile']
        print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
        print(num_requests['message'])
        path = num_requests['shape_path']
        file = num_requests['shapefile_input']
        gid = ''
        if env == 'prod':
            gid = num_requests['prod_gid']
        elif env == 'uat' or env == 'sit':
            gid = num_requests['uat_gid']
        data = {'format': num_requests['outfile_format'],'granuleID': gid}
        files = {
            'shapefile': (
            file,
            open(path + file,'rb'),
            num_requests['post_type']
            ),
            }
        r = requests.post(harmony_url, files=files, data=data)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext[0:] == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_alos_l22_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    elif file_ext[2:] == 'nc':
        product_info = get_nc_info.get_nc_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_alos_l22_nc_' + num_requests['q_num'] + '.py'
        #write_the_testfile = write_testfile.write_testfile(filename, collection, product_info, num_requests)
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)   
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_alos_l22_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)        
        
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)

Running request 1: https://harmony.uat.earthdata.nasa.gov/C1239927797-ASF/ogc-api-coverages/1.0.0/collections/MASK%2CINC/coverage/rangeset?subset=lat(75.11:75.19)&subset=lon(111.35:111.85)&format=image%2Ftiff&granuleID=G1239969288-ASF
Subset by bbox. Variable subset bands MASK & INC
Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_alos_l22_1.py::test_alos_l22_1_cs PASSED                            [  9%]
test_alos_l22_1.py::test_alos_l22_1_projection PASSED                    [ 18%]
test_alos_l22_1.py::test_alos_l22_1_proj_epsg PASSED                     [ 27%]
test

Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_alos_l22_nc_5.py::test_alos_l22_5_cs PASSED                         [  9%]
test_alos_l22_nc_5.py::test_alos_l22_5_projection PASSED                 [ 18%]
test_alos_l22_nc_5.py::test_alos_l22_5_proj_epsg PASSED                  [ 27%]
test_alos_l22_nc_5.py::test_alos_l22_5_gcs PASSED                        [ 36%]
test_alos_l22_nc_5.py::test_alos_l22_5_gcs_epsg PASSED                   [ 45%]
test_alos_l22_nc_5.py::test_alos_l22_5_authority PASSED                  [ 54%]
test_alos_l22_nc_5.py::test_alos_l22_5_subset PA

## Regression tests for the S1 InSAR (Beta) collection:

In [3]:
import requests
import s1_info
import get_base_url_info
import get_product_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
    
collection = 's1_insar'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
s1 = s1_info.s1_info()
# Loop through the list of dictionaries in s1_info.py
for num_requests in s1:    
    harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
    outfile = env + num_requests['outfile']
    print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
    print(num_requests['message'])
    r = requests.get(harmony_url)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_grfn_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_grfn_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)

Running request 1: https://harmony.uat.earthdata.nasa.gov/C1225776654-ASF/ogc-api-coverages/1.0.0/collections/science%2Fgrids%2Fdata%2Famplitude/coverage/rangeset?subset=lat(33:33.1)&subset=lon(-115.5:-115.25)&format=image%2Ftiff&granuleID=G1234646236-ASF
Variable subset amplitude and spatial subset.
Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_grfn_1.py::test_s1_insar_1_cs PASSED                                [  9%]
test_grfn_1.py::test_s1_insar_1_projection PASSED                        [ 18%]
test_grfn_1.py::test_s1_insar_1_proj_epsg PASSED                  

Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_grfn_5.py::test_s1_insar_5_cs PASSED                                [  9%]
test_grfn_5.py::test_s1_insar_5_projection PASSED                        [ 18%]
test_grfn_5.py::test_s1_insar_5_proj_epsg PASSED                         [ 27%]
test_grfn_5.py::test_s1_insar_5_gcs PASSED                               [ 36%]
test_grfn_5.py::test_s1_insar_5_gcs_epsg PASSED                          [ 45%]
test_grfn_5.py::test_s1_insar_5_authority PASSED                         [ 54%]
test_grfn_5.py::test_s1_insar_5_subset PASSED   

Status code:  200  Success!
============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/tstern/.local/share/virtualenvs/gdal_subsetter_regression-nY2L-x8q/bin/python
cachedir: .pytest_cache
rootdir: /home/tstern/repos/asf-gdal-subsetter/asf-harmony-gdal/Notebooks/gdal_subsetter_regression
collected 11 items                                                             

test_grfn_9.py::test_s1_insar_9_cs PASSED                                [  9%]
test_grfn_9.py::test_s1_insar_9_projection PASSED                        [ 18%]
test_grfn_9.py::test_s1_insar_9_proj_epsg PASSED                         [ 27%]
test_grfn_9.py::test_s1_insar_9_gcs PASSED                               [ 36%]
test_grfn_9.py::test_s1_insar_9_gcs_epsg PASSED                          [ 45%]
test_grfn_9.py::test_s1_insar_9_authority PASSED                         [ 54%]
test_grfn_9.py::test_s1_insar_9_subset PASSED   

test_grfn_11.py::test_s1_insar_11_filetype PASSED                        [ 20%]
test_grfn_12.py::test_s1_insar_12_filetype PASSED                        [ 21%]
test_grfn_2.py::test_s1_insar_2_cs PASSED                                [ 22%]
test_grfn_2.py::test_s1_insar_2_projection PASSED                        [ 23%]
test_grfn_2.py::test_s1_insar_2_proj_epsg PASSED                         [ 24%]
test_grfn_2.py::test_s1_insar_2_gcs PASSED                               [ 25%]
test_grfn_2.py::test_s1_insar_2_gcs_epsg PASSED                          [ 25%]
test_grfn_2.py::test_s1_insar_2_authority PASSED                         [ 26%]
test_grfn_2.py::test_s1_insar_2_subset PASSED                            [ 27%]
test_grfn_2.py::test_s1_insar_2_bands PASSED                             [ 28%]
test_grfn_2.py::test_s1_insar_2_variables PASSED                         [ 29%]
test_grfn_2.py::test_s1_insar_2_xy_size PASSED                           [ 30%]
test_grfn_2.py::test_s1_insar_2_identica

## Regression tests for the UAVSAR POLSAR PAULI collection:
### Not currently working in SIT or UAT

In [ ]:
import requests
import uavsar_info
import get_base_url_info
import get_product_info
import compare_images
import write_testfile
import get_filetype
import write_filetype_test
import pytest
import os
import cleanup
import glob

if 'env' not in globals():
    global env
    env = 'uat'
collection = 'uavsar'
base_url_info = get_base_url_info.get_base_url_info(env,collection)
uavsar = uavsar_info.uavsar_info()
for num_requests in uavsar:    
    harmony_url = base_url_info['base_url'] + base_url_info['collection_id'] + '/ogc-api-coverages/1.0.0/collections/' + num_requests['request_url'] + num_requests[base_url_info['granule_env_flag']]
    outfile = env + num_requests['outfile']
    print("Running request " + num_requests['q_num'] + ': ' + harmony_url)
    print(num_requests['message'])
    r = requests.get(harmony_url)
    status_code = r.status_code    
    if status_code == 200:
        print ('Status code: ',status_code, ' Success!')
    else:
        print (('Status code: ',status_code, ' Something went wrong!'))
    with open(outfile, 'wb') as f:
        f.write(r.content)
        f.close()
    file_ext = outfile[-4:]
    if  file_ext == 'tiff':
        product_info = get_product_info.get_product_info(collection, outfile)
        is_identical = compare_images.compare_images(num_requests['reference_image'],outfile)
        filename = 'test_uavsar_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_testfile.write_testfile(filename, collection, is_identical, product_info, num_requests)
        ! pytest -v -p no:warnings
    else:
        filetype = get_filetype.get_filetype(outfile)
        filename = 'test_uavsar_' + num_requests['q_num'] + '.py'
        write_the_testfile = write_filetype_test.write_filetype_test(filename, collection, filetype, num_requests)
        ! pytest -v -p no:warnings
    # Keep the test file around for later
    if os.path.exists(filename):
        os.rename(filename, 'temp'+filename)
    else:
        print("The file " + filename + " does not exist")
    # Get rid of the product file
    clean = cleanup.cleanup(outfile)
# Rename the temp files to test files
files = glob.glob('temp*')
for f in files:
    old_name = f
    new_name=f.replace('temp', '')
    try:
        os.rename(f, new_name)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))
# Run all the tests one more time to get the cumulative result
! pytest -v -p no:warnings
# Get rid of the test files
files = glob.glob('test*')
for f in files:
    clean = cleanup.cleanup(f)